In [1]:
import os
import json
import pickle
import requests
import collections
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
with open('data/jeopardy_358974_questions_20190612.pkl', 'rb') as f:
    karl_to_question_id = pickle.load(f).to_dict()['question_id']
with open('data/jeopardy_310326_question_player_pairs_20190612.pkl', 'rb') as f:
    records_df = pickle.load(f)
    
def get_result(card):
    record_id = karl_to_question_id[int(card['question_id'])]
    prob = (records_df[records_df.question_id == record_id]['correct'] / 2 + 0.5).mean()
    return 'correct' if prob > 0.5 else 'wrong'

In [ ]:
with open('data/diagnostic_questions.pkl', 'rb') as f:
    cards = pickle.load(f)
    
shown_cards = []
results = []
while len(shown_cards) < 300:
    r = requests.post('http://127.0.0.1:8000/api/karl/schedule', data=json.dumps(cards))
    r = json.loads(r.text)
    show_card = cards[r['order'][0]]
    show_card['label'] = get_result(show_card)
    shown_cards.append(show_card)
    results.append(1 if show_card['label'] == 'correct' else 0)
    print('{: <40}{: <20}{: <20}{: <20}'.format(
        show_card['answer'],
        show_card['category'],
        show_card['label'],
        np.mean(results)
    ))
    print(r['rationale'])
    print()
    r = requests.post('http://127.0.0.1:8000/api/karl/update', data=json.dumps([show_card]))
    cards = [x for x in cards if x['question_id'] != show_card['question_id']]

In [5]:
import pickle
from scheduler import MovingAvgScheduler

In [3]:
with open('data/diagnostic_questions.pkl', 'rb') as f:
    cards = pickle.load(f)

In [6]:
scheduler = MovingAvgScheduler(db_filename='db_test.sqlite')

loading question and records...
merging dfs...
scheduler ready


/fs/clip-quiz/shifeng/anaconda/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator CountVectorizer from version 0.21.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/fs/clip-quiz/shifeng/anaconda/lib/python3.7/site-packages/sklearn/base.py:306: UserWarning: Trying to unpickle estimator LatentDirichletAllocation from version 0.21.2 when using version 0.21.3. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [8]:
qrep = scheduler.embed_one(cards[0])
print(qrep)
skill = np.zeros_like(qrep)
print(skill, skill.size)
print(np.argmax(qrep))
skill[np.argmax(qrep)] = 1
skill *= scheduler.predict_one(cards[0])

[0.00833333 0.00833333 0.00833333 0.00833333 0.00833333 0.00833333
 0.00833333 0.00833333 0.00833333 0.00833333 0.00833333 0.00833333
 0.00833333 0.00833333 0.00833333 0.00833333 0.00833333 0.00833333
 0.00833333 0.84166667]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 20
19
